This cell block is for importing all the required libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import levene
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy.stats import ttest_ind

In [14]:
# Specify the path to your CSV file on Google Drive (replace with your actual path)
data_path = 'C:\Users\SAMEER\All Form Data.csv'

# Read the data from the CSV file
data = pd.read_csv(data_path)

This cell block contains code to perfoerm the ANOVA test

In [15]:
# Fit the ANOVA model with ScoreChange as the target variable and TeachingMethod as the grouping variable:
model = smf.ols("ChangeInScore ~ C(Gender)", data=data).fit()

# Perform ANOVA with type 2 tests for mixed effects:
results = anova_lm(model, typ=2)

# Print the results:
print(results)

               sum_sq    df         F    PR(>F)
C(Gender)    1.238095   1.0  0.098016  0.757634
Residual   240.000000  19.0       NaN       NaN


In [ ]:
# Separate data for each teaching method
teaching_method1_data = data[data['TeachingMethod'] == 'Application']['ScoreChange']
teaching_method2_data = data[data['TeachingMethod'] == 'Human Tutor']['ScoreChange']
teaching_method3_data = data[data['TeachingMethod'] == 'NAO']['ScoreChange']

# Levene's test (replace with actual method names)
w_statistic, p_value = levene(teaching_method1_data, teaching_method2_data, teaching_method3_data)

print(f"Levene's test statistic: {w_statistic}")
print(f"p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("Fail to reject the null hypothesis of equal variances.")
else:
    print("Reject the null hypothesis of equal variances.")

Levene's test statistic: 0.8913738019169329
p-value: 0.4274379268498227
Fail to reject the null hypothesis of equal variances.


In [ ]:
# Separate features (including the group variable) and the target variable
features = data.drop('ScoreChange', axis=1)
target_variable = data['ScoreChange']

groups = np.unique(features['TeachingMethod'])  # Get unique group labels

# Perform pairwise t-tests with Bonferroni correction
for i in range(len(groups) - 1):
  for j in range(i + 1, len(groups)):
    _, p_value = ttest_ind(target_variable[features['TeachingMethod'] == groups[i]],
                           target_variable[features['TeachingMethod'] == groups[j]])
    alpha_adjusted = 0.05 / (len(groups) * (len(groups) - 1) / 2)  # Bonferroni correction
    if p_value < alpha_adjusted:
      print(f"Significant difference between {groups[i]} and {groups[j]} (p-value: {p_value:.4f})")
    else:
      print(p_value)
      print(alpha_adjusted)

0.295539594516705
0.016666666666666666
0.0839899482776256
0.016666666666666666
0.6032164494297924
0.016666666666666666
